<a href="https://colab.research.google.com/github/AUNitman/test_ml_classifier/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gdown

In [ ]:
from google.colab import drive
from google.colab import files

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# to_csv('/content/drive/MyDrive/Проекты/Titanic_kaggle/clean_train.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Проекты/Classifier of X-ray images/train.csv')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 2124
